In [1]:
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path

In [2]:
indir = "/scratch/jhm3ab/antibiotic_bii/filtered_antibiotics"
filename = 'all_antibiotics.json'
#for filename in os.listdir(indir):
fin = os.path.join(indir, filename)
f = open(fin, 'r')
data = json.load(f)
f.close()

In [3]:
class_data = {}
ndcdir = "/scratch/jhm3ab/antibiotic_bii/antibiotic_class_ndc"
for classfile in os.listdir(ndcdir):
    if classfile.endswith(".txt"):
        cin = os.path.join(ndcdir, classfile)
        f = open(cin, 'r')
        drug_list = f.read().splitlines()
        c = Path(classfile).stem
        class_data[c] = []
        member_zip = pd.DataFrame()
        provider_zip = pd.DataFrame()
        for time in data:
            month_data = data[time]
            member_month = pd.Series(dtype = 'int', name = time)
            provider_month = pd.Series(dtype = 'int', name = time)
            for drug in month_data:
                if drug in drug_list:
                    frame = pd.DataFrame.from_dict(month_data[drug])
                    m_counts = frame['Member Zip Code DOS'].value_counts()
                    p_counts = frame['Service Provider ZIP'].value_counts()
                    if member_month.empty:
                        member_month = m_counts
                        member_month.rename(time, inplace = True)
                    else:
                        member_month.add(m_counts, fill_value=0)
                    if provider_month.empty:
                        provider_month = p_counts
                        provider_month.rename(time, inplace = True)
                    else:
                        provider_month.add(p_counts, fill_value=0)
            member_zip = pd.concat([member_zip, member_month], axis = 1).fillna(0)
            provider_zip = pd.concat([provider_zip, provider_month], axis = 1).fillna(0)
        class_data[c].append(member_zip.to_dict())
        class_data[c].append(provider_zip.to_dict())

In [4]:
with open("antibiotics_analysis/class_counts.json", "w") as f:
    json.dump(class_data, f)